## General idea: Find out whether the genetics influence the microbiome by comparing the samples within the monozygotic and dizygotic pairs and find the significance of the beta diversity. 

In [8]:
# importing all required packages & notebook extensions at the start of the notebook
import os
import pandas as pd
import qiime2 as q2
from qiime2 import Visualization
import matplotlib.pyplot as plt
%matplotlib inline
from operator import itemgetter
import matplotlib.patches as mpatches
from scipy.stats import shapiro
from os.path import exists

or_dir = '../data' #original data (demux sequences, metadata)
data_dir = 'data' #data from polybox (ASV, taxonomy analysis)

In [3]:
! wget -nv -O data.zip 'https://polybox.ethz.ch/index.php/s/pNA39R0rl2xMMj9/download'
! unzip -q data.zip #-d $data_dir
! mv data data2
! mv data2/taxonomy/data .
! cp data2/ASV/data/* data
! cp data2/Phylogeny/data/* data
! rm -r data2
! rm data.zip

2022-11-27 10:36:30 URL:https://polybox.ethz.ch/index.php/s/pNA39R0rl2xMMj9/download [903525744] -> "data.zip" [1]


### 1. Separate metadata table for mono- and dizygotic twins and generate a table for each twin individually. (Or maybe for each pair?)

In [2]:
metadata = pd.read_csv(or_dir + '/metadata.tsv', sep = '\t')
host_numbers = metadata['host_id'].unique()
host_numbers

array([42.1, 27.2, 28.1, 28.2, 39.2,  8.1,  8.2, 29.1, 40.1, 40.2, 35.1,
       35.2, 47.1, 47.2,  4.1,  4.2, 29.2,  3.1, 30.2, 36.1, 36.2,  6.1,
        6.2, 30.1, 33.1, 33.2, 43.2, 44.1, 44.2, 45.1, 45.2,  5.1, 37.1,
       37.2, 39.1, 46.1,  3.2, 43.1, 42.2, 46.2,  5.2, 27.1, 48.2, 48.1,
       32.1, 32.2, 12.2, 13.2, 14.1, 14.2, 10.1, 10.2, 12.1, 13.1, 15.1,
       15.2, 16.1, 25.1, 25.2, 26.2, 11.1,  2.1,  2.2, 20.1, 20.2, 21.1,
       21.2, 23.1, 23.2, 19.2, 16.2, 17.1, 17.2, 18.1, 18.2, 19.1, 24.2,
       11.2, 24.1, 26.1])

In [16]:
all_hosts = dict()
for host in host_numbers: #loop through all unique host ids
    #print(host)
    new_name = 'df_host_'+str(host)
    #new_name = new_name.replace('.', '_')
    all_hosts[host] = metadata[metadata['host_id']==host]
    locals()[new_name] = metadata[metadata['host_id']==host]

### 2. Problem: some samples contain NaN values, but the host has been weaned before. We need to keep those values and assign the status of weaned and lose all others that do not contain any information. 

In [53]:
metadata[metadata['host_id']==23.1].sort_values(by=['collection_date'])

,id,Library Layout,Instrument,collection_date,geo_location_name,geo_latitude,geo_longitude,host_id,age_days,weight_kg,...,birth_length_cm,sex,delivery_mode,zygosity,race,ethnicity,delivery_preterm,diet_milk,diet_weaning,age_months
1600,ERR1311612,PAIRED,Illumina MiSeq,2010-06-09 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,23.1,36.0,NaN,...,47.0,male,Vaginal,Dizygotic,Caucasian,Not Hispanic,True,fd,False,1.0
1589,ERR1311616,PAIRED,Illumina MiSeq,2010-07-08 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,23.1,65.0,4.763,...,47.0,male,Vaginal,Dizygotic,Caucasian,Not Hispanic,True,fd,False,2.0
1258,ERR1310030,PAIRED,Illumina MiSeq,2010-11-10 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,23.1,190.0,6.804,...,47.0,male,Vaginal,Dizygotic,Caucasian,Not Hispanic,True,fd,True,6.0
1259,ERR1310031,PAIRED,Illumina MiSeq,2010-12-03 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,23.1,213.0,NaN,...,47.0,male,Vaginal,Dizygotic,Caucasian,Not Hispanic,True,fd,True,7.0
904,ERR1310681,PAIRED,Illumina MiSeq,2011-01-14 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,23.1,256.0,NaN,...,47.0,male,Vaginal,Dizygotic,Caucasian,Not Hispanic,True,NaN,NaN,8.0
905,ERR1310682,PAIRED,Illumina MiSeq,2011-02-17 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,23.1,290.0,NaN,...,47.0,male,Vaginal,Dizygotic,Caucasian,Not Hispanic,True,NaN,NaN,10.0
906,ERR1310683,PAIRED,Illumina MiSeq,2011-03-16 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,23.1,316.0,NaN,...,47.0,male,Vaginal,Dizygotic,Caucasian,Not Hispanic,True,fd,True,10.0
1617,ERR1311611,PAIRED,Illumina MiSeq,2011-05-05 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,23.1,366.0,NaN,...,47.0,male,Vaginal,Dizygotic,Caucasian,Not Hispanic,True,fd,True,12.0
1587,ERR1311614,PAIRED,Illumina MiSeq,2011-07-07 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,23.1,428.0,NaN,...,47.0,male,Vaginal,Dizygotic,Caucasian,Not Hispanic,True,NaN,NaN,14.0
1586,ERR1311613,PAIRED,Illumina MiSeq,2011-08-05 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,23.1,457.0,NaN,...,47.0,male,Vaginal,Dizygotic,Caucasian,Not Hispanic,True,NaN,NaN,15.0


#### The NaN values appear to only be after weaning or when not weaned at all, but not before weaning. (How do we check whether that is really true?)

#### If so, we can assign True to each NaN value if metadata['diet_weaning'].sum() >= 1:
#### (We discussed this with our tutor and she gave permission to use this assumption)

In [5]:
for host in host_numbers:
    if metadata['diet_weaning'].sum() >= 1:
        metadata['diet_weaning'].fillna(True)

#### Check:

In [21]:
all_hosts[16.1].sort_values(by=['collection_date'])

,id,Library Layout,Instrument,collection_date,geo_location_name,geo_latitude,geo_longitude,host_id,age_days,weight_kg,...,birth_length_cm,sex,delivery_mode,zygosity,race,ethnicity,delivery_preterm,diet_milk,diet_weaning,age_months
879,ERR1310621,PAIRED,Illumina MiSeq,2010-05-06 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,16.1,47.0,3.487,...,42.0,female,Cesarean,Dizygotic,Caucasian,Not Hispanic,True,fd,False,2.0
837,ERR1310614,PAIRED,Illumina MiSeq,2010-07-09 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,16.1,111.0,NaN,...,42.0,female,Cesarean,Dizygotic,Caucasian,Not Hispanic,True,fd,True,4.0
1553,ERR1311558,PAIRED,Illumina MiSeq,2010-08-02 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,16.1,135.0,NaN,...,42.0,female,Cesarean,Dizygotic,Caucasian,Not Hispanic,True,fd,False,4.0
838,ERR1310615,PAIRED,Illumina MiSeq,2010-09-11 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,16.1,175.0,NaN,...,42.0,female,Cesarean,Dizygotic,Caucasian,Not Hispanic,True,fd,True,6.0
1386,ERR1309946,PAIRED,Illumina MiSeq,2010-10-09 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,16.1,203.0,NaN,...,42.0,female,Cesarean,Dizygotic,Caucasian,Not Hispanic,True,fd,True,7.0
840,ERR1310617,PAIRED,Illumina MiSeq,2010-11-05 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,16.1,230.0,NaN,...,42.0,female,Cesarean,Dizygotic,Caucasian,Not Hispanic,True,fd,True,8.0
839,ERR1310616,PAIRED,Illumina MiSeq,2010-11-24 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,16.1,249.0,NaN,...,42.0,female,Cesarean,Dizygotic,Caucasian,Not Hispanic,True,fd,True,8.0
841,ERR1310618,PAIRED,Illumina MiSeq,2011-01-05 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,16.1,291.0,NaN,...,42.0,female,Cesarean,Dizygotic,Caucasian,Not Hispanic,True,fd,True,10.0
1385,ERR1309945,PAIRED,Illumina MiSeq,2011-02-04 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,16.1,322.0,NaN,...,42.0,female,Cesarean,Dizygotic,Caucasian,Not Hispanic,True,fd,True,11.0
1387,ERR1309947,PAIRED,Illumina MiSeq,2011-02-26 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,16.1,343.0,NaN,...,42.0,female,Cesarean,Dizygotic,Caucasian,Not Hispanic,True,fd,True,11.0


### 3. Get table for each twin pair and each stage

#### Pair the twins:

In [3]:
pair_numbers = list(dict.fromkeys(host_numbers.round(0)))

In [7]:
all_pairs = dict()
for pair in pair_numbers: #loop through all unique pair ids
    a = metadata[metadata['host_id'] == pair+0.1]
    b = metadata[metadata['host_id'] == pair+0.2]
    all_pairs[pair] = pd.concat([a,b])

In [ ]:
#why was this necessary in the previous dictionary loop?
#new_name = 'df_pair_'+str(pair)
#locals()[new_name] = metadata[metadata['host_id']==host]

#### Get tables for each pair and each stage:

In [8]:
stages = metadata['diet_milk'].dropna().unique()
stages

all_stages = dict()

for stage in stages:
    new_name = 'df_stage_'+str(stage)
    all_stages[stage] = metadata[metadata['diet_milk'] == stage]
    locals()[new_name] = metadata[metadata['diet_milk'] == stage]

#### Check:

In [61]:
all_stages['fd']

,id,Library Layout,Instrument,collection_date,geo_location_name,geo_latitude,geo_longitude,host_id,age_days,weight_kg,...,birth_length_cm,sex,delivery_mode,zygosity,race,ethnicity,delivery_preterm,diet_milk,diet_weaning,age_months
0,ERR1314182,PAIRED,Illumina MiSeq,2011-11-11 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,42.1,232.0,NaN,...,47.0,male,Cesarean,Monozygotic,Caucasian,Not Hispanic,True,fd,True,8.0
1,ERR1314183,PAIRED,Illumina MiSeq,2010-12-11 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,27.2,192.0,NaN,...,45.0,female,Cesarean,Dizygotic,Caucasian,Hispanic,True,fd,True,6.0
8,ERR1314190,PAIRED,Illumina MiSeq,2012-02-12 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,40.1,324.0,NaN,...,49.0,female,Vaginal,Monozygotic,Caucasian,Not Hispanic,True,fd,True,11.0
9,ERR1314191,PAIRED,Illumina MiSeq,2012-02-12 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,40.2,325.0,NaN,...,44.0,female,Vaginal,Monozygotic,Caucasian,Not Hispanic,True,fd,True,11.0
13,ERR1314198,PAIRED,Illumina MiSeq,2012-04-12 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,47.1,305.0,NaN,...,46.0,male,Vaginal,Monozygotic,African-American,Not Hispanic,True,fd,True,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1661,ERR1310702,PAIRED,Illumina MiSeq,2011-04-12 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,25.2,335.0,9.696,...,48.0,female,Cesarean_emergency,Monozygotic,Caucasian,Hispanic,False,fd,True,11.0
1664,ERR1310705,PAIRED,Illumina MiSeq,2010-07-31 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,26.1,74.0,NaN,...,49.0,female,Vaginal,Dizygotic,Caucasian,Not Hispanic,False,fd,False,2.0
1665,ERR1310707,PAIRED,Illumina MiSeq,2011-02-07 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,27.1,250.0,NaN,...,45.0,female,Cesarean,Dizygotic,Caucasian,Hispanic,True,fd,True,8.0
1666,ERR1310708,PAIRED,Illumina MiSeq,2011-04-09 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,27.1,310.0,NaN,...,45.0,female,Cesarean,Dizygotic,Caucasian,Hispanic,True,fd,True,10.0


#### In order to generate a table that contains values of two dictionaries with separate conditions, we can merge them like this:

In [62]:
all_stages['fd'].merge(all_pairs[40], how = 'inner')

,id,Library Layout,Instrument,collection_date,geo_location_name,geo_latitude,geo_longitude,host_id,age_days,weight_kg,...,birth_length_cm,sex,delivery_mode,zygosity,race,ethnicity,delivery_preterm,diet_milk,diet_weaning,age_months
0,ERR1314190,PAIRED,Illumina MiSeq,2012-02-12 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,40.1,324.0,NaN,...,49.0,female,Vaginal,Monozygotic,Caucasian,Not Hispanic,True,fd,True,11.0
1,ERR1314191,PAIRED,Illumina MiSeq,2012-02-12 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,40.2,325.0,NaN,...,44.0,female,Vaginal,Monozygotic,Caucasian,Not Hispanic,True,fd,True,11.0
2,ERR1314250,PAIRED,Illumina MiSeq,2011-10-12 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,40.1,200.0,NaN,...,49.0,female,Vaginal,Monozygotic,Caucasian,Not Hispanic,True,fd,True,7.0
3,ERR1314054,PAIRED,Illumina MiSeq,2011-11-09 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,40.1,229.0,NaN,...,49.0,female,Vaginal,Monozygotic,Caucasian,Not Hispanic,True,fd,True,8.0
4,ERR1314055,PAIRED,Illumina MiSeq,2011-11-09 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,40.2,230.0,NaN,...,44.0,female,Vaginal,Monozygotic,Caucasian,Not Hispanic,True,fd,True,8.0
5,ERR1314557,PAIRED,Illumina MiSeq,2011-09-14 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,40.2,173.0,NaN,...,44.0,female,Vaginal,Monozygotic,Caucasian,Not Hispanic,True,fd,True,6.0
6,ERR1314295,PAIRED,Illumina MiSeq,2011-09-13 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,40.1,172.0,NaN,...,49.0,female,Vaginal,Monozygotic,Caucasian,Not Hispanic,True,fd,True,6.0
7,ERR1314716,PAIRED,Illumina MiSeq,2011-08-15 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,40.1,144.0,NaN,...,49.0,female,Vaginal,Monozygotic,Caucasian,Not Hispanic,True,fd,True,5.0
8,ERR1314717,PAIRED,Illumina MiSeq,2011-08-15 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,40.2,143.0,NaN,...,44.0,female,Vaginal,Monozygotic,Caucasian,Not Hispanic,True,fd,True,5.0
9,ERR1315635,PAIRED,Illumina MiSeq,2011-07-29 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,40.2,126.0,5.273,...,44.0,female,Vaginal,Monozygotic,Caucasian,Not Hispanic,True,fd,False,4.0


#### That way we can generate a new dictionary with which we can recall all pairs with a certain diet:

In [9]:
pair_stages = dict()

for pair in pair_numbers:
    for stage in stages:
        pair_stages[pair,stage] = all_stages[stage].merge(all_pairs[pair], how = 'inner')

#### Check:

In [10]:
pair_stages[3, 'fd']

,id,Library Layout,Instrument,collection_date,geo_location_name,geo_latitude,geo_longitude,host_id,age_days,weight_kg,...,birth_length_cm,sex,delivery_mode,zygosity,race,ethnicity,delivery_preterm,diet_milk,diet_weaning,age_months
0,ERR1314203,PAIRED,Illumina MiSeq,2010-06-12 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,3.1,151.0,NaN,...,48.0,male,Vaginal,Monozygotic,Caucasian,Not Hispanic,True,fd,True,5.0
1,ERR1314448,PAIRED,Illumina MiSeq,2010-12-13 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,3.2,336.0,NaN,...,49.0,male,Vaginal,Monozygotic,Caucasian,Not Hispanic,True,fd,True,11.0
2,ERR1314828,PAIRED,Illumina MiSeq,2010-04-17 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,3.1,95.0,NaN,...,48.0,male,Vaginal,Monozygotic,Caucasian,Not Hispanic,True,fd,False,3.0
3,ERR1314888,PAIRED,Illumina MiSeq,2010-04-18 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,3.2,96.0,NaN,...,49.0,male,Vaginal,Monozygotic,Caucasian,Not Hispanic,True,fd,False,3.0
4,ERR1314844,PAIRED,Illumina MiSeq,2010-06-17 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,3.2,156.0,NaN,...,49.0,male,Vaginal,Monozygotic,Caucasian,Not Hispanic,True,fd,True,5.0
5,ERR1314860,PAIRED,Illumina MiSeq,2010-11-17 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,3.1,309.0,NaN,...,48.0,male,Vaginal,Monozygotic,Caucasian,Not Hispanic,True,fd,True,10.0
6,ERR1314640,PAIRED,Illumina MiSeq,2010-05-15 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,3.2,123.0,6.606,...,49.0,male,Vaginal,Monozygotic,Caucasian,Not Hispanic,True,fd,True,4.0
7,ERR1315089,PAIRED,Illumina MiSeq,2010-12-21 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,3.1,344.0,NaN,...,48.0,male,Vaginal,Monozygotic,Caucasian,Not Hispanic,True,fd,True,11.0
8,ERR1315188,PAIRED,Illumina MiSeq,2010-09-24 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,3.2,256.0,NaN,...,49.0,male,Vaginal,Monozygotic,Caucasian,Not Hispanic,True,fd,True,8.0
9,ERR1315190,PAIRED,Illumina MiSeq,2010-10-24 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,3.2,286.0,NaN,...,49.0,male,Vaginal,Monozygotic,Caucasian,Not Hispanic,True,fd,True,9.0


### 4. Filter feature table according to metadata table

In [ ]:
! qiime feature-table filter-samples --help

In [4]:
meta_wean = pd.read_csv(data_dir + '/metadata_only_before_after_weaning.csv')

In [5]:
meta_wean.to_csv('meta_wean.tsv', sep='\t', index = False)

#### Weaned:

In [4]:
for pair in pair_numbers:
    pair1, pair2 = pair + 0.1, pair + 0.2
     
    ! qiime feature-table filter-samples \
        --i-table $data_dir/phylogeny_filtered_table.qza \
        --m-metadata-file $data_dir/meta_wean.tsv \
        --p-where "[diet_weaning]='weaned' and ([host_id]=$pair1 or [host_id]=$pair2)" \
        --o-filtered-table $data_dir/pairs_weaned/pair_"$pair"_weaned.qza

Saved FeatureTable[Frequency] to: data/pairs_weaned/pair_42.0_weaned.qza
Saved FeatureTable[Frequency] to: data/pairs_weaned/pair_27.0_weaned.qza
Saved FeatureTable[Frequency] to: data/pairs_weaned/pair_28.0_weaned.qza
Saved FeatureTable[Frequency] to: data/pairs_weaned/pair_39.0_weaned.qza
Saved FeatureTable[Frequency] to: data/pairs_weaned/pair_8.0_weaned.qza
Saved FeatureTable[Frequency] to: data/pairs_weaned/pair_29.0_weaned.qza
Saved FeatureTable[Frequency] to: data/pairs_weaned/pair_40.0_weaned.qza
Saved FeatureTable[Frequency] to: data/pairs_weaned/pair_35.0_weaned.qza
Saved FeatureTable[Frequency] to: data/pairs_weaned/pair_47.0_weaned.qza
Saved FeatureTable[Frequency] to: data/pairs_weaned/pair_4.0_weaned.qza
Saved FeatureTable[Frequency] to: data/pairs_weaned/pair_3.0_weaned.qza
Saved FeatureTable[Frequency] to: data/pairs_weaned/pair_30.0_weaned.qza
Saved FeatureTable[Frequency] to: data/pairs_weaned/pair_36.0_weaned.qza
Saved FeatureTable[Frequency] to: data/pairs_weaned/pa

#### Not Weaned:

In [5]:
for pair in pair_numbers:
    pair1, pair2 = pair + 0.1, pair + 0.2
    
    ! qiime feature-table filter-samples \
    --i-table $data_dir/phylogeny_filtered_table.qza \
    --m-metadata-file $data_dir/meta_wean.tsv \
    --p-where "[diet_weaning]='not weaning' and ([host_id]=$pair1 or [host_id]=$pair2)" \
    --o-filtered-table $data_dir/pairs_notweaned/pair_"$pair"_notweaned.qza

Saved FeatureTable[Frequency] to: data/pairs_notweaned/pair_42.0_notweaned.qza
Saved FeatureTable[Frequency] to: data/pairs_notweaned/pair_27.0_notweaned.qza
Saved FeatureTable[Frequency] to: data/pairs_notweaned/pair_28.0_notweaned.qza
Saved FeatureTable[Frequency] to: data/pairs_notweaned/pair_39.0_notweaned.qza
Saved FeatureTable[Frequency] to: data/pairs_notweaned/pair_8.0_notweaned.qza
Saved FeatureTable[Frequency] to: data/pairs_notweaned/pair_29.0_notweaned.qza
Saved FeatureTable[Frequency] to: data/pairs_notweaned/pair_40.0_notweaned.qza
Saved FeatureTable[Frequency] to: data/pairs_notweaned/pair_35.0_notweaned.qza
Saved FeatureTable[Frequency] to: data/pairs_notweaned/pair_47.0_notweaned.qza
Saved FeatureTable[Frequency] to: data/pairs_notweaned/pair_4.0_notweaned.qza
Saved FeatureTable[Frequency] to: data/pairs_notweaned/pair_3.0_notweaned.qza
Saved FeatureTable[Frequency] to: data/pairs_notweaned/pair_30.0_notweaned.qza
Saved FeatureTable[Frequency] to: data/pairs_notweaned/

In [10]:
 ! qiime feature-table filter-samples --help

Usage: qiime feature-table filter-samples [OPTIONS]

  Filter samples from table based on frequency and/or metadata. Any features
  with a frequency of zero after sample filtering will also be removed. See
  the filtering tutorial on https://docs.qiime2.org for additional details.

Inputs:
  --i-table ARTIFACT FeatureTable[Frequency¹ | RelativeFrequency² |
    PresenceAbsence³ | Composition⁴]
                       The feature table from which samples should be
                       filtered.                                    [required]
Parameters:
  --p-min-frequency INTEGER
                       The minimum total frequency that a sample must have to
                       be retained.                               [default: 0]
  --p-max-frequency INTEGER
                       The maximum total frequency that a sample can have to
                       be retained. If no value is provided this will default
                       to infinity (i.e., no maximum frequency filter will 

### 5. Find F values for twin column with ANCOM showing differences between individuals

#### For the ANCOM I need the host ID column as categorical instead of numeric.

In [9]:
! qiime tools cast-metadata --help

Usage: qiime tools cast-metadata [OPTIONS] METADATA...

  Designate metadata column types. Supported column types are as follows:
  categorical, numeric. Providing multiple file paths to this command will
  merge the metadata.

Options:
  --cast COLUMN:TYPE  Parameter for each metadata column that should be cast
                      as a specified column type (supported types are as
                      follows: categorical, numeric). The required formatting
                      for this parameter is --cast COLUMN:TYPE, repeated for
                      each column and the associated column type it should be
                      cast to in the output.                        [required]
  --ignore-extra      If this flag is enabled, cast parameters that do not
                      correspond to any of the column names within the
                      provided metadata will be ignored.
  --error-on-missing  If this flag is enabled, failing to include cast
                      param

In [6]:
! qiime tools cast-metadata $data_dir/meta_wean.tsv \
--cast host_id:categorical \
--output-file $data_dir/meta_wean2.tsv

#### For Weaned:

In [7]:
for pair in pair_numbers:
    ! qiime composition add-pseudocount \
    --i-table $data_dir/pairs_weaned/pair_"$pair"_weaned.qza \
    --o-composition-table $data_dir/pairs_weaned/pair_"$pair"_weaned_pseudo.qza

Saved FeatureTable[Composition] to: data/pairs_weaned/pair_42.0_weaned_pseudo.qza
Saved FeatureTable[Composition] to: data/pairs_weaned/pair_27.0_weaned_pseudo.qza
Saved FeatureTable[Composition] to: data/pairs_weaned/pair_28.0_weaned_pseudo.qza
Saved FeatureTable[Composition] to: data/pairs_weaned/pair_39.0_weaned_pseudo.qza
Saved FeatureTable[Composition] to: data/pairs_weaned/pair_8.0_weaned_pseudo.qza
Saved FeatureTable[Composition] to: data/pairs_weaned/pair_29.0_weaned_pseudo.qza
Saved FeatureTable[Composition] to: data/pairs_weaned/pair_40.0_weaned_pseudo.qza
Saved FeatureTable[Composition] to: data/pairs_weaned/pair_35.0_weaned_pseudo.qza
Saved FeatureTable[Composition] to: data/pairs_weaned/pair_47.0_weaned_pseudo.qza
Saved FeatureTable[Composition] to: data/pairs_weaned/pair_4.0_weaned_pseudo.qza
Saved FeatureTable[Composition] to: data/pairs_weaned/pair_3.0_weaned_pseudo.qza
Saved FeatureTable[Composition] to: data/pairs_weaned/pair_30.0_weaned_pseudo.qza
Saved FeatureTable[

In [12]:
for pair in pair_numbers:
    ! qiime composition ancom \
    --i-table $data_dir/pairs_weaned/pair_"$pair"_weaned_pseudo.qza \
    --m-metadata-file $data_dir/meta_wean2.tsv \
    --m-metadata-column host_id \
    --p-transform-function log \
    --o-visualization $data_dir/ancoms_weaned/ancom_"$pair"_weaned.qzv

Saved Visualization to: data/ancoms_weaned/ancom_42.0_weaned.qzv
Saved Visualization to: data/ancoms_weaned/ancom_27.0_weaned.qzv
Saved Visualization to: data/ancoms_weaned/ancom_28.0_weaned.qzv
Saved Visualization to: data/ancoms_weaned/ancom_39.0_weaned.qzv
Saved Visualization to: data/ancoms_weaned/ancom_8.0_weaned.qzv
Plugin error from composition:

  `ids_to_keep` must contain at least one ID.

Debug info has been saved to /tmp/qiime2-q2cli-err-yn1ve37p.log
Saved Visualization to: data/ancoms_weaned/ancom_40.0_weaned.qzv
Plugin error from composition:

  `ids_to_keep` must contain at least one ID.

Debug info has been saved to /tmp/qiime2-q2cli-err-qundbbo_.log
Plugin error from composition:

  `ids_to_keep` must contain at least one ID.

Debug info has been saved to /tmp/qiime2-q2cli-err-19o_hdlm.log
Plugin error from composition:

  `ids_to_keep` must contain at least one ID.

Debug info has been saved to /tmp/qiime2-q2cli-err-49h5olvy.log
Saved Visualization to: data/ancoms_wea

#### For Not Weaned:

In [11]:
for pair in pair_numbers:
    ! qiime composition add-pseudocount \
    --i-table $data_dir/pairs_notweaned/pair_"$pair"_notweaned.qza \
    --o-composition-table $data_dir/pairs_notweaned/pair_"$pair"_notweaned_pseudo.qza

Saved FeatureTable[Composition] to: data/pairs_notweaned/pair_42.0_notweaned_pseudo.qza
Saved FeatureTable[Composition] to: data/pairs_notweaned/pair_27.0_notweaned_pseudo.qza
Saved FeatureTable[Composition] to: data/pairs_notweaned/pair_28.0_notweaned_pseudo.qza
Saved FeatureTable[Composition] to: data/pairs_notweaned/pair_39.0_notweaned_pseudo.qza
Saved FeatureTable[Composition] to: data/pairs_notweaned/pair_8.0_notweaned_pseudo.qza
Saved FeatureTable[Composition] to: data/pairs_notweaned/pair_29.0_notweaned_pseudo.qza
Saved FeatureTable[Composition] to: data/pairs_notweaned/pair_40.0_notweaned_pseudo.qza
Saved FeatureTable[Composition] to: data/pairs_notweaned/pair_35.0_notweaned_pseudo.qza
Saved FeatureTable[Composition] to: data/pairs_notweaned/pair_47.0_notweaned_pseudo.qza
Saved FeatureTable[Composition] to: data/pairs_notweaned/pair_4.0_notweaned_pseudo.qza
Saved FeatureTable[Composition] to: data/pairs_notweaned/pair_3.0_notweaned_pseudo.qza
Saved FeatureTable[Composition] to:

In [4]:
for pair in pair_numbers:
    ! qiime composition ancom \
    --i-table $data_dir/pairs_notweaned/pair_"$pair"_notweaned_pseudo.qza \
    --m-metadata-file $data_dir/meta_wean2.tsv \
    --m-metadata-column host_id \
    --p-transform-function log \
    --o-visualization $data_dir/ancoms_notweaned/ancom_"$pair"_notweaned.qzv

Saved Visualization to: data/ancoms_notweaned/ancom_42.0_notweaned.qzv
Saved Visualization to: data/ancoms_notweaned/ancom_27.0_notweaned.qzv
Plugin error from composition:

  All values in `grouping` are unique. This method cannot operate on a grouping vector with only unique values (e.g., there are no 'within' variance because each group of samples contains only a single sample).

Debug info has been saved to /tmp/qiime2-q2cli-err-6t_8myfz.log
Plugin error from composition:

  All values in `grouping` are unique. This method cannot operate on a grouping vector with only unique values (e.g., there are no 'within' variance because each group of samples contains only a single sample).

Debug info has been saved to /tmp/qiime2-q2cli-err-t1bpvox3.log
Saved Visualization to: data/ancoms_notweaned/ancom_8.0_notweaned.qzv
Plugin error from composition:

  `ids_to_keep` must contain at least one ID.

Debug info has been saved to /tmp/qiime2-q2cli-err-j120i6s5.log
Saved Visualization to: data/

#### Some twins do not have any samples for the stages 'weaned' or 'not weaning' leading to the error messages.

In [6]:
!qiime tools export \
  --input-path $data_dir/ancoms_notweaned/ancom_24.0_notweaned.qzv \
  --output-path $data_dir/ancoms_notweaned/ancom_24.0_notweaned.tsv

Exported data/ancoms_notweaned/ancom_24.0_notweaned.qzv as Visualization to directory data/ancoms_notweaned/ancom_24.0_notweaned.tsv


In [8]:
Visualization.load(f'{data_dir}/ancoms_notweaned/ancom_24.0_notweaned.qzv')

<visualization: Visualization uuid: 05201a39-a4c7-425f-adb7-175c79bc5133>

#### Now we have the the volcano plot and corresponding table showing the W-values for all different features. In order to make further statistical analysis, we define a threshold for the W-values and count the features above. This number reflects the difference between the twins for the corresponding stage. 

#### For this, we need to convert the ancom results to a tsv file:

#### For weaned:

In [4]:
for pair in pair_numbers:
    !qiime tools export \
      --input-path $data_dir/ancoms_weaned/ancom_"$pair"_weaned.qzv \
      --output-path $data_dir/ancoms_weaned/ancom_"$pair"_weaned.tsv

Exported data/ancoms_weaned/ancom_42.0_weaned.qzv as Visualization to directory data/ancoms_weaned/ancom_42.0_weaned.tsv
Exported data/ancoms_weaned/ancom_27.0_weaned.qzv as Visualization to directory data/ancoms_weaned/ancom_27.0_weaned.tsv
Exported data/ancoms_weaned/ancom_28.0_weaned.qzv as Visualization to directory data/ancoms_weaned/ancom_28.0_weaned.tsv
Exported data/ancoms_weaned/ancom_39.0_weaned.qzv as Visualization to directory data/ancoms_weaned/ancom_39.0_weaned.tsv
Exported data/ancoms_weaned/ancom_8.0_weaned.qzv as Visualization to directory data/ancoms_weaned/ancom_8.0_weaned.tsv
Usage: qiime tools export [OPTIONS]

  Exporting extracts (and optionally transforms) data stored inside an
  Artifact or Visualization. Note that Visualizations cannot be transformed
  with --output-format

Options:
  --input-path ARTIFACT/VISUALIZATION
                        Path to file that should be exported        [required]
  --output-path PATH    Path to file or directory where data sh

#### For not weaned:

In [5]:
for pair in pair_numbers:
    !qiime tools export \
      --input-path $data_dir/ancoms_notweaned/ancom_"$pair"_notweaned.qzv \
      --output-path $data_dir/ancoms_notweaned/ancom_"$pair"_notweaned.tsv

Exported data/ancoms_notweaned/ancom_42.0_notweaned.qzv as Visualization to directory data/ancoms_notweaned/ancom_42.0_notweaned.tsv
Exported data/ancoms_notweaned/ancom_27.0_notweaned.qzv as Visualization to directory data/ancoms_notweaned/ancom_27.0_notweaned.tsv
Usage: qiime tools export [OPTIONS]

  Exporting extracts (and optionally transforms) data stored inside an
  Artifact or Visualization. Note that Visualizations cannot be transformed
  with --output-format

Options:
  --input-path ARTIFACT/VISUALIZATION
                        Path to file that should be exported        [required]
  --output-path PATH    Path to file or directory where data should be
                        exported to                                 [required]
  --output-format TEXT  Format which the data should be exported as. This
                        option cannot be used with Visualizations
  --help                Show this message and exit.

                    There was a problem with the command:

#### Next, we export the ANCOM results of each tsv to pandas so we can do further calculations:

#### Generate new dictionaries with the significant ASVs:

In [15]:
ancoms_weaned = dict()
for pair in pair_numbers:
    file_exists = exists(f'{data_dir}/ancoms_weaned/ancom_'+str(pair)+'_weaned.tsv')
    if file_exists == True:
        ancom = pd.read_csv(data_dir + '/ancoms_weaned/ancom_'+str(pair)+'_weaned.tsv/ancom.tsv', sep = '\t')
        ancoms_weaned[pair] = ancom[ancom['Reject null hypothesis']==True]

In [19]:
ancoms_notweaned = dict()
for pair in pair_numbers:
    file_exists = exists(f'{data_dir}/ancoms_notweaned/ancom_'+str(pair)+'_notweaned.tsv')
    if file_exists == True:
        ancom = pd.read_csv(data_dir + '/ancoms_notweaned/ancom_'+str(pair)+'_notweaned.tsv/ancom.tsv', sep = '\t')
        ancoms_notweaned[pair] = ancom[ancom['Reject null hypothesis']==True]

#### For the next step we need to find out which numbers belong to the mono- or dizygotic twin pairs:

In [22]:
monos = metadata['host_id'][metadata['zygosity']=='Monozygotic']
monozygotic_ids = list(dict.fromkeys(monos.round(0)))
dis = metadata['host_id'][metadata['zygosity']=='Dizygotic']
dizygotic_ids = list(dict.fromkeys(dis.round(0)))

#### Then we can count the amount of significantly differentially abundant ASVs for the weaned and not weaned stages.

#### First just for mono- and dizygotic:

In [55]:
mono_numbers = dict()
for pair in monozygotic_ids:
    existence_weaned = exists(f'{data_dir}/ancoms_weaned/ancom_'+str(pair)+'_weaned.tsv')
    existence_notweaned = exists(f'{data_dir}/ancoms_notweaned/ancom_'+str(pair)+'_notweaned.tsv')
    if existence_weaned == True:
        number_weaned = len(ancoms_weaned[pair].index)
    if existence_notweaned == True:
        number_notweaned = len(ancoms_notweaned[pair].index)
    mono_numbers[pair] = number_weaned + number_notweaned

In [56]:
di_numbers = dict()
for pair in dizygotic_ids:
    existence_weaned = exists(f'{data_dir}/ancoms_weaned/ancom_'+str(pair)+'_weaned.tsv')
    existence_notweaned = exists(f'{data_dir}/ancoms_notweaned/ancom_'+str(pair)+'_notweaned.tsv')
    if existence_weaned == True:
        number_weaned = len(ancoms_weaned[pair].index)
    if existence_notweaned == True:
        number_notweaned = len(ancoms_notweaned[pair].index)
    di_numbers[pair] = number_weaned + number_notweaned

#### And also for each stage:

In [46]:
mono_weaned = dict()
for pair in monozygotic_ids:
    existence_weaned = exists(f'{data_dir}/ancoms_weaned/ancom_'+str(pair)+'_weaned.tsv')
    if existence_weaned == True:
        number_weaned = len(ancoms_weaned[pair].index)
    mono_weaned[pair] = number_weaned

In [47]:
mono_notweaned = dict()
for pair in monozygotic_ids:
    existence_notweaned = exists(f'{data_dir}/ancoms_notweaned/ancom_'+str(pair)+'_notweaned.tsv')
    if existence_notweaned == True:
        number_notweaned = len(ancoms_notweaned[pair].index)
    mono_notweaned[pair] = number_notweaned

In [49]:
di_weaned = dict()
for pair in dizygotic_ids:
    existence_weaned = exists(f'{data_dir}/ancoms_weaned/ancom_'+str(pair)+'_weaned.tsv')
    if existence_weaned == True:
        number_weaned = len(ancoms_weaned[pair].index)
    di_weaned[pair] = number_weaned

In [50]:
di_notweaned = dict()
for pair in dizygotic_ids:
    existence_notweaned = exists(f'{data_dir}/ancoms_notweaned/ancom_'+str(pair)+'_notweaned.tsv')
    if existence_notweaned == True:
        number_notweaned = len(ancoms_notweaned[pair].index)
    di_notweaned[pair] = number_notweaned

#### Get numbers into dataframe:

#### Just mono- and dizygotic:

In [57]:
mono_diff = pd.DataFrame.from_dict(mono_numbers, orient='index')
mono_diff.mean()

0    13.571429
dtype: float64

In [58]:
di_diff = pd.DataFrame.from_dict(di_numbers, orient='index')
di_diff.mean()

0    77.777778
dtype: float64

#### And also depending on stage:

In [51]:
mono_diff_weaned = pd.DataFrame.from_dict(mono_weaned, orient='index')
mono_diff_weaned.mean()

0    1.619048
dtype: float64

In [52]:
mono_diff_notweaned = pd.DataFrame.from_dict(mono_notweaned, orient='index')
mono_diff_notweaned.mean()

0    11.952381
dtype: float64

In [53]:
di_diff_weaned = pd.DataFrame.from_dict(di_weaned, orient='index')
di_diff_weaned.mean()

0    19.166667
dtype: float64

In [54]:
di_diff_notweaned = pd.DataFrame.from_dict(di_notweaned, orient='index')
di_diff_notweaned.mean()

0    58.611111
dtype: float64

#### Now we have the two dataframes for each zygosity. As index we have the number of the twin pair and next to it the number of ASVs for which we could reject the null hypothesis in the ANCOM (significantly differentially abundant).

# How can the means within the stages be smaller than when just comparing zygosity?

### 6. Beta diversity for visuals

In [7]:
! qiime diversity alpha-rarefaction \
    --i-table $data_dir/phylogeny_filtered_table.qza \
    --i-phylogeny $data_dir/reference-tree.qza \
    --p-max-depth 10000 \
    --m-metadata-file $or_dir/new_metadata_for_a_diversity.tsv \
    --o-visualization $data_dir/alpha-rarefaction2.qzv

Saved Visualization to: data/alpha-rarefaction2.qzv


In [8]:
Visualization.load(f'{data_dir}/alpha-rarefaction2.qzv')

<visualization: Visualization uuid: b35ac1ff-5d92-4097-adf1-7cb5a01826d8>

#### Rarefaction at 1500

In [9]:
! qiime diversity core-metrics-phylogenetic \
  --i-table $data_dir/phylogeny_filtered_table.qza \
  --i-phylogeny $data_dir/reference-tree.qza \
  --m-metadata-file $or_dir/metadata.tsv \
  --p-sampling-depth 1500 \
  --output-dir $data_dir/core-metrics-results

Saved FeatureTable[Frequency] to: data/core-metrics-results/rarefied_table.qza
Saved SampleData[AlphaDiversity] to: data/core-metrics-results/faith_pd_vector.qza
Saved SampleData[AlphaDiversity] to: data/core-metrics-results/observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: data/core-metrics-results/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: data/core-metrics-results/evenness_vector.qza
Saved DistanceMatrix to: data/core-metrics-results/unweighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: data/core-metrics-results/weighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: data/core-metrics-results/jaccard_distance_matrix.qza
Saved DistanceMatrix to: data/core-metrics-results/bray_curtis_distance_matrix.qza
Saved PCoAResults to: data/core-metrics-results/unweighted_unifrac_pcoa_results.qza
Saved PCoAResults to: data/core-metrics-results/weighted_unifrac_pcoa_results.qza
Saved PCoAResults to: data/core-metrics-results/jaccard_pcoa_results.qza
Sa

In [16]:
Visualization.load(f'{data_dir}/core-metrics-results/unweighted_unifrac_emperor.qzv')

<visualization: Visualization uuid: 431fa773-66d2-42e4-ac40-f65f5139c66d>

In [17]:
Visualization.load(f'{data_dir}/core-metrics-results/weighted_unifrac_emperor.qzv')

<visualization: Visualization uuid: 67705828-583e-45ac-84b4-81304be6e651>

### 7. ANCOM for zygosity column --> find significance